In [1]:
%load_ext autoreload
%autoreload 2

In [72]:
import os 
from time import sleep
from multiprocessing import Queue, Manager, Process

from functions import yt_dl, pth_vids, pth_vid
from master_functions import iter0, sfm_pipe 

In [19]:
def master_iter0(args):

    """Performs iter0() and launch sfm_pipe processes with 
    the generated sets."""
    
    path_set_dir, width = iter0(*args)
    pth_sets = [os.path.join(path_set_dir, el) for el in os.listdir(path_set_dir)]
    # start new process
    for pth_set in pth_sets:
        args = (pth_set, width)
        Process(target=sfm_pipe, args=args).start()

def launching(rate, f_frms, f_ftrs, f_mtchs, sample, plylsts=[], vids=[]):
    
    for plylst in plylsts:
        for v_id in os.listdir(pth_plylst(plylst)):
            args = (pth_vid(v_id, plylst), rate,f_frms, f_ftrs, f_mtchs, sample)
            Process(target=master_iter0, args=args).start()

    for v_id in vids:
        args = (pth_vid(v_id), rate, f_frms, f_ftrs, f_mtchs, sample)
        Process(target=master_iter0, args=(args,)).start()
                  
def drain(rate, f_frms, f_ftrs, f_mtchs, sample, plylsts=[], vids=[]):
    
    print('Start Draining')
    execution = Process(target=launching, args=(rate, f_frms, f_ftrs, f_mtchs, sample, plylsts, vids))
    execution.start()
    execution.join()
    print('Finish Draining')

In [20]:
rate = 2
sample = True 
f_frms  = f_ftrs = f_mtchs = True
PLYLST = []

VIDS = os.listdir(pth_vids())

drain(rate, f_frms, f_ftrs, f_mtchs, sample,
      plylsts=PLYLST, vids=VIDS)

Start Draining


Process Process-6:2:
Traceback (most recent call last):
  File "/Users/xrubiato/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/xrubiato/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-19-b49d0d5958e8>", line 6, in master_iter0
    path_set_dir, width = iter0(*args)
  File "/Users/xrubiato/DHLab/Draining-Youtube/master_functions.py", line 42, in iter0
    xtrct_frame(v_id, plylst, sample, rate)
  File "/Users/xrubiato/DHLab/Draining-Youtube/functions.py", line 189, in xtrct_frame
    make_dir(path_frames)
  File "/Users/xrubiato/DHLab/Draining-Youtube/functions.py", line 42, in make_dir
    os.mkdir(pth_dir)
FileNotFoundError: [Errno 2] No such file or directory: '/Users/xrubiato/DHLab/Draining-Youtube/videos/.DS_Store/frames'
Process Process-6:1:1:
Traceback (most recent call last):
  File "/Users/xrubiato/anaconda3/lib/python3.6/multipro

Finish Draining


In [ ]:
yt_dl('https://www.youtube.com/watch?v=b3dMHvxGIKY')

### Sequential 

In [75]:
import youtube_dl as youtube

In [77]:
youtube

<module 'youtube_dl' from '/home/dhacker/anaconda3/lib/python3.7/site-packages/youtube_dl/__init__.py'>

In [85]:
from __future__ import unicode_literals
import youtube_dl

ydl_opts = {'preferredqaulity':'248'}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=v3dclL2grbs'])

[youtube] v3dclL2grbs: Downloading webpage
[youtube] v3dclL2grbs: Downloading video info webpage


[download] Destination: Samsung Demo Video  (Display Auflösung Test) 720p [HD]-v3dclL2grbs.f136.mp4
[download] 100% of 10.13MiB in 00:0002MiB/s ETA 00:00known ETA
[download] Destination: Samsung Demo Video  (Display Auflösung Test) 720p [HD]-v3dclL2grbs.f251.webm
[download] 100% of 1016.63KiB in 00:0060MiB/s ETA 00:00known ETA
[ffmpeg] Merging formats into "Samsung Demo Video  (Display Auflösung Test) 720p [HD]-v3dclL2grbs.mkv"
Deleting original file Samsung Demo Video  (Display Auflösung Test) 720p [HD]-v3dclL2grbs.f136.mp4 (pass -k to keep)
Deleting original file Samsung Demo Video  (Display Auflösung Test) 720p [HD]-v3dclL2grbs.f251.webm (pass -k to keep)


In [74]:
from multiprocessing import Manager, Process

from functions import pth_vid, yt_dl
from master_functions import iter0, sfm_pipe


In [73]:
def iter0_seq(args_iter0, shared_list):
    """Calls iter= and append the return to the shared_list"""
    paths_sets, width = iter0(*args_iter0)
    for pth in paths_sets:
        shared_list.append((pth, width))


def launching_seq_aux(target, args_l):
    """ Function used to launch Processes in parallel"""
    for args in args_l:
        Process(target=target, args=args).start()


def launching_seq(vids_list, rates=1, cpu_nbr=8, f_frms=True, f_ftrs=True, f_mtchs=True, sample=False):
    """Function that spawns at maximum a number of processes equal to the parameter cpu_nbr."""
    manager = Manager()
    shared_list = manager.list()

    # Repeat until all vids have been processed

    target = iter0_seq
    while not vids_list.empty():

        args_l=[]
        if len(vids_list) >= cpu_nbr:
            for i in range(cpu_nbr):
                args_iter0 = (pth_vid(v_id=vids_list.pop), rates[-i-1], f_frms, f_ftrs, f_mtchs, sample)
                args_l.append((args_iter0, shared_list))

        else: # less videos than cpu_numbers
            for i in range(len(vids_list)):
                args_l.append((pth_vid(v_id=vids_list.pop), rates[-i - 1], f_frms, f_ftrs, f_mtchs, sample))

        control_launch = Process(target=launching_seq_aux, args=(target, args_l))
        control_launch.start()
        control_launch.join()

    # Finish iter0 part
    target = sfm_pipe
    while not sfm_pipe_list.empty():

        args_l = []
        if len(shared_list) >= cpu_nbr:
            for i in range(cpu_nbr):
                args_l.append(shared_list.pop())

        else: # less sfm_pipe remaining than cpu_number

            for i in range(len(shared_list)):
                args_l.append(shared_list.pop())

        control_launch = Process(target=launching_seq_aux, args=(target, args_l))
        control_launch.start()
        control_launch.join()

In [ ]:

VIDS = [u]

### Test 

In [70]:
l1 = [0,1,2,3,4,5,6,7,8,9]
l2 = ['test{}'.format(i) for i in range(len(l1))]

for i in range(5):  
    print(l1.pop())
    print(l2[-(i+1)])
print('\n')
    
for idx, el in enumerate(l1):
    print(el)
    print(l2[idx])

9
test9
8
test8
7
test7
6
test6
5
test5


0
test0
1
test1
2
test2
3
test3
4
test4


9
test9
8
test8
7
test7
6
test6
5
test5


In [4]:
p = Process(target=f1, args=())
p.start()
p.join()

NameError: name 'Process' is not defined

## Archive 

In [ ]:
class Worker:

    def __init__(self, Q):
        self.Q = Q


    def Qiter0(self, args):
        """Performs iter0() and enqueues an sfm_pipe process with the generated sets."""
        path_set_dir, width = iter0(*args)
        pth_sets = [os.path.join(path_set_dir, el) for el in os.listdir(path_set_dir)]
        print(width)
        # Enqueue next tasks
        for pth_set in pth_sets:
            args = (pth_set, width)
            self.Q.put(('sfm_pipe', args))


    def sfm_pipe(self, args):
        sfm_pipe(*args)


    def work(self, func, args):

        if func == 'Qiter0':
            self.Qiter0(args)

        elif func == 'sfm_pipe':
            self.sfm_pipe(args)

            
def worker_exec(Q, wip_l):
    
    func, args = Q.get()
    # append to wip list 
    wip_l.append(func)
    
    # instanciate worker 
    worker = Worker(Q)
    worker.work(func, args)
    # remove from list after it finishes 
    wip_l.remove(func)

    
def drain(rate, f_frms, f_ftrs, f_mtchs, sample,
           plylsts=[], vids=[]):
    
    Q = Queue()
    manager = Manager()
    wip_l = manager.list()
    
    for plylst in plylsts:
        for v_id in os.listdir(pth_plylst(plylst)):
            
            args = (pth_vid(v_id, plylst), rate, f_frms, f_ftrs, f_mtchs, sample)
            Q.put(('Qiter0', args))

    for v_id in vids:
            
            args = (pth_vid(v_id), rate, f_frms, f_ftrs, f_mtchs, sample)
            Q.put(('Qiter0', args))
                  
    while True:
        
        if not Q.empty():
            Process(target=worker_exec, args=(Q, wip_l)).start()
            sleep(0.5)
        
        else: 
            if len(wip_l) == 0:
                print('Finish Draining')
                break 